### Import Dependencies

In [31]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent.parent / "PromptWizard"))

import promptwizard
from promptwizard.glue.promptopt.instantiate import GluePromptOpt
from promptwizard.glue.promptopt.techniques.common_logic import DatasetSpecificProcessing
from promptwizard.glue.common.utils.file import save_jsonlist
from typing import Any
from tqdm import tqdm
from re import compile, findall
import os
from datasets import load_dataset

from dotenv import load_dotenv
load_dotenv(override = True)

True

### Create a dataset specific class and define the required functions 

In [45]:
import json
import pandas as pd
from sklearn.model_selection import train_test_split

# Load full data
df = pd.read_json("../../../data/results.jsonl", lines=True)

# Create groundtruth judgment
df['final_answer'] = "false"
positive = df['sample_class'] == 'positive'
df.loc[positive & (df['question_id'] == df['bge_retrieved_top1_id']), 'final_answer'] = "true"

# Filter only train split
df_train = df[df['split'] == 'train'].reset_index(drop=True)

# Select and rename
pw_df = df_train[['query_id', 'question_id', 'user_query', 'bge_retrieved_top1_id', 'final_answer']].copy()
pw_df = pw_df.rename(columns={'bge_retrieved_top1_id': 'retrieved_id'})

# Load D1 passages
id_to_qa = {}
with open("../../../data/D1.jsonl", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        id_to_qa[obj['id']] = {'candidate_Q': obj['Q'], 'candidate_A': obj['A']}

# Add candidate Q&A
pw_df['candidate_Q'] = pw_df['retrieved_id'].map(lambda x: id_to_qa.get(x, {}).get('candidate_Q', '[Missing]'))
pw_df['candidate_A'] = pw_df['retrieved_id'].map(lambda x: id_to_qa.get(x, {}).get('candidate_A', '[Missing]'))

# Add formatted Input column
pw_df['question'] = (
    "User Query: " + pw_df['user_query'] + "\n" +
    "Retrieved Question: " + pw_df['candidate_Q'] + "\n" +
    "Retrieved Answer: " + pw_df['candidate_A']
)

# Final column order (Input first for clarity)
pw_df = pw_df[['question', 'query_id', 'question_id', 'user_query', 'retrieved_id',
               'candidate_Q', 'candidate_A', 'final_answer']]

# Re-split after adding column
train_pw, test_pw = train_test_split(pw_df, train_size=50, test_size=30, random_state=42, shuffle=True)
train_pw = train_pw.reset_index(drop=True)
test_pw = test_pw.reset_index(drop=True)

# Save
train_pw.to_json("data/train.jsonl", orient="records", lines=True)
test_pw.to_json("data/test.jsonl", orient="records", lines=True)

print(f"Train: {len(train_pw)}, Test: {len(test_pw)}")

Train: 50, Test: 30


In [47]:
class PromptWizardProcessing(DatasetSpecificProcessing):
    def dataset_to_jsonl(self, dataset_jsonl: str, **kwargs: Any) -> None:
        def extract_answer_from_output(completion):
            return completion

        examples_set = []
        examples_set = []
        for _, sample in tqdm(enumerate(kwargs["dataset"]), desc="Converting samples"):
            example = {
                DatasetSpecificProcessing.QUESTION_LITERAL           : sample["question"],
                DatasetSpecificProcessing.ANSWER_WITH_REASON_LITERAL : sample["answer"],
                DatasetSpecificProcessing.FINAL_ANSWER_LITERAL        : extract_answer_from_output(sample["answer"])
            }
            examples_set.append(example)
        save_jsonlist(dataset_jsonl, examples_set, "w")

    def extract_final_answer(self, answer: str):
        if not answer:
            return self.INVALID_ANS
        answer = answer.strip().lower()
        if "true" in answer or "yes" in answer:
            return "true"
        elif "false" in answer or "no" in answer:
            return "false"
        else:
            return self.INVALID_ANS

In [48]:
judge_processor = PromptWizardProcessing()

### Set paths

In [49]:
train_file_name = os.path.join("data", "train.jsonl")
test_file_name = os.path.join("data", "test.jsonl")
path_to_config = "configs"
promptopt_config_path = os.path.join(path_to_config, "promptopt_config.yaml")
setup_config_path = os.path.join(path_to_config, "setup_config.yaml")

### Create an object for calling prompt optimization and inference functionalities

In [50]:
gp = GluePromptOpt(promptopt_config_path,
                   setup_config_path,
                   train_file_name,
                   judge_processor)

Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o-mini')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'retrieval_judgment'), ('mode', 'offline')] 


Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o-mini')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'retrieval_judgment'), ('mode', 'offline')] 


Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o-mini')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'retrieval_judgment'), ('mode', 'offline')] 


Setup configurations parameters: [('assistant_llm', AssistantLLM(prompt_opt='gpt-4o-mini')), ('description', None), ('dir_info', Dir(base_dir='logs', log_dir_name='glue_logs')), ('experiment_name', 'retrieval_judgment'), ('mode', 'offline')] 


Prompt Optimization 

### Call prompt optmization function
1. ```use_examples``` can be used when there are training samples and a mixture of real and synthetic in-context examples are required in the final prompt. When set to ```False``` all the in-context examples will be real
2. ```generate_synthetic_examples``` can be used when there are no training samples and we want to generate synthetic examples 
3. ```run_without_train_examples``` can be used when there are no training samples and in-context examples are not required in the final prompt 

In [51]:
# Function call to generate optimal prompt and expert profile 
best_prompt, expert_profile = gp.get_best_prompt(use_examples=True,run_without_train_examples=False,generate_synthetic_examples=False)


Mutating Task Description....


Iterations completed:   0%|          | 0/3 [00:00<?, ?it/s]
 + Starting iteration: 1 
 current_base_instruction: Carefully judge the semantic and intention of both user query and the retrieved QA, decide whether the retrieved question exactly match with the user intent. Note that there is high compliance regulation, be caution with the decision.

 + Starting iteration: 1 
 current_base_instruction: Carefully judge the semantic and intention of both user query and the retrieved QA, decide whether the retrieved question exactly match with the user intent. Note that there is high compliance regulation, be caution with the decision.

 + Starting iteration: 1 
 current_base_instruction: Carefully judge the semantic and intention of both user query and the retrieved QA, decide whether the retrieved question exactly match with the user intent. Note that there is high compliance regulation, be caution with the decision.

 + Starting iteration: 1 
 current_base_instruction: Carefully judge the 

dataset_subset [{'question': 'User Query: I am trying to determine the normal yearly wage scale for a Concierge position within Diamond Resorts International?\nRetrieved Question: salary for a concierge with diamond international\nRetrieved Answer: The average Diamond Resorts International salary ranges from approximately $20,000 per year for Concierge. However, the average hourly pay for a Concierge at Diamond Resorts International ranges from approximately $9.00 per hour to $40.00 per hour.', 'query_id': '8628_p5', 'question_id': 8628, 'user_query': 'I am trying to determine the normal yearly wage scale for a Concierge position within Diamond Resorts International?', 'retrieved_id': 8628, 'candidate_Q': 'salary for a concierge with diamond international', 'candidate_A': 'The average Diamond Resorts International salary ranges from approximately $20,000 per year for Concierge. However, the average hourly pay for a Concierge at Diamond Resorts International ranges from approximately $9

prompt_score_list [["You are a strict service chatbot judge. Your task is to strictly determine whether a retrieved QA is the true response for the given user query. Answer only 'Yes' or 'No' without explanation.\nCarefully judge the semantic and intention of both user query and the retrieved QA, decide whether the retrieved question exactly match with the user intent. Note that there is high compliance regulation, be caution with the decision.", 1.0, [{'question': "User Query: What is the founding date of Ubisoft's branch in Quebec?\nRetrieved Question: When was Ubisoft Quebec founded?\nRetrieved Answer: Ubisoft Quebec was founded in 2005 in Quebec City, Quebec.", 'query_id': '5033_p0', 'question_id': 5033, 'user_query': "What is the founding date of Ubisoft's branch in Quebec?", 'retrieved_id': 5033, 'candidate_Q': 'When was Ubisoft Quebec founded?', 'candidate_A': 'Ubisoft Quebec was founded in 2005 in Quebec City, Quebec.', 'final_answer': 'true'}]], ['Carefully analyze the user qu

dataset_subset [{'question': 'User Query: Would focusing on preventing illness rather than responding to it enhance our health outcomes?\nRetrieved Question: Would we be healthier if we switched from a reactive healthcare system to preventative?\nRetrieved Answer: Switching from a reactive healthcare system to a preventative one could potentially lead to improved overall health and a reduction in healthcare costs. Prevention focuses on maintaining good health, preventing diseases and enhancing wellbeing by addressing underlying risk factors and promoting healthy behaviors . By investing in preventative care, individuals may experience fewer chronic diseases, reduced hospitalizations, and less need for advanced medical treatments . Preventative healthcare also reduces healthcare costs by preventing the onset or progression of diseases, which are typically more expensive to treat than to prevent . However, it is important to recognize that a balance between reactive and preventative heal

...

100%|██████████| 3/3 [00:22<00:00,  7.58s/it]



Generating Expert Identity....


Expert Identity: [Agent Description]: You are a meticulous and impartial evaluator with expertise in assessing the accuracy and relevance of question-and-answer pairs. You have a sharp eye for detail and a strong understanding of how to match user queries with the correct responses. You are highly disciplined in following strict criteria and do not allow personal opinions or assumptions to influence your judgment. Your role is to make definitive determinations, answering only 'Yes' or 'No,' based solely on whether the retrieved answer truly satisfies the user’s query. You excel at maintaining consistency, fairness, and precision in all evaluations.
Expert Identity: [Agent Description]: You are a meticulous and impartial evaluator with expertise in assessing the accuracy and relevance of question-and-answer pairs. You have a sharp eye for detail and a strong understanding of how to match user queries with the correct responses. You are highly disciplined in following strict criteria and


Generating Intent Keywords....


Final best prompt:   
You are a meticulous QA evaluation agent. Your task is to determine if a retrieved question-answer pair fully resolves the user’s query. Consider whether the QA pair addresses all parts of the query and maintains semantic equivalence. Think about how you could measure whether the response makes progress toward fully answering the user’s question. Respond only with 'Yes' or 'No'.  


[Question] User Query: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average?
Retrieved Question: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average salary for concierges?
Retrieved Answer: The average yearly salary for a Concierge at Diamond Resorts International is approximately $20,000 per year, which is below the industry average of $25,000 for concierges in similar resorts.
[Answer] [Improved Reasoning Chain]:  

1. **Identi

### Save the optimized prompt and expert profile

In [52]:
import pickle 

if not os.path.exists("results"):
    os.system("mkdir results")
    
with open("results/best_prompt.pkl", 'wb') as f:
    pickle.dump(best_prompt, f)
with open("results/expert_profile.pkl", 'wb') as f:
    pickle.dump(expert_profile, f)

print(f"Best prompt: {best_prompt} \nExpert profile: {expert_profile}")

Best prompt:   
You are a meticulous QA evaluation agent. Your task is to determine if a retrieved question-answer pair fully resolves the user’s query. Consider whether the QA pair addresses all parts of the query and maintains semantic equivalence. Think about how you could measure whether the response makes progress toward fully answering the user’s question. Respond only with 'Yes' or 'No'.  


[Question] User Query: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average?
Retrieved Question: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average salary for concierges?
Retrieved Answer: The average yearly salary for a Concierge at Diamond Resorts International is approximately $20,000 per year, which is below the industry average of $25,000 for concierges in similar resorts.
[Answer] [Improved Reasoning Chain]:  

1. **Identify the

In [53]:
import os
import pickle

os.makedirs("results", exist_ok=True)

# Save as pickle
with open("results/best_prompt.pkl", 'wb') as f:
    pickle.dump(best_prompt, f)
with open("results/expert_profile.pkl", 'wb') as f:
    pickle.dump(expert_profile, f)

# Save as readable .txt
with open("results/best_prompt.txt", "w", encoding="utf-8") as f:
    f.write(best_prompt)

with open("results/expert_profile.txt", "w", encoding="utf-8") as f:
    f.write(expert_profile)

print("Saved best_prompt and expert_profile as both .pkl and .txt in ./results/")
print("\n=== BEST PROMPT ===\n")
print(best_prompt)
print("\n=== EXPERT PROFILE ===\n")
print(expert_profile)

Saved best_prompt and expert_profile as both .pkl and .txt in ./results/

=== BEST PROMPT ===

  
You are a meticulous QA evaluation agent. Your task is to determine if a retrieved question-answer pair fully resolves the user’s query. Consider whether the QA pair addresses all parts of the query and maintains semantic equivalence. Think about how you could measure whether the response makes progress toward fully answering the user’s question. Respond only with 'Yes' or 'No'.  


[Question] User Query: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average?
Retrieved Question: What is the average yearly salary for a concierge at Diamond Resorts International, and how does it compare to the industry average salary for concierges?
Retrieved Answer: The average yearly salary for a Concierge at Diamond Resorts International is approximately $20,000 per year, which is below the industry average of $25,000 for concie

In [24]:
expert_profile

'You are a data scientist with a strong background in machine learning, particularly in natural language processing and unsupervised learning techniques. You have expertise in text analysis and clustering algorithms, enabling you to effectively group sentences based on their semantic similarities. Your skills include applying methods such as k-means clustering, hierarchical clustering, and advanced techniques like topic modeling or sentence embeddings to create meaningful clusters. You possess a deep understanding of the nuances of language and contextual variations, ensuring that your clustering takes into account both syntactical and semantic content. By leveraging your expertise, you can output exactly {n} cluster labels, ensuring each sentence is accurately classified into one of the {k} clusters. Your analytical prowess ensures that the clustering solution is both interpretable and robust, aligning with the underlying patterns of the data.'

### Evaluate the optimized prompt

In [22]:
gp.EXPERT_PROFILE = expert_profile
gp.BEST_PROMPT = best_prompt

# Function call to evaluate the prompt
accuracy = gp.evaluate(test_file_name)

print(f"Final Accuracy: {accuracy}")

Evaluation started 

Evaluation started 





#################### Running 




{'accuracy': '0/1 : 0.0%', 'predicted': '[7, 19, 14, 5, 12, 9, 8, 13, 18, 15, 1, 6, 11, 8, 16, 17, 4, 10]', 'actual': '[37, 133, 80, 142, 100, 20, 125, 28, 147, 94, 82, 17, 101, 43, 92, 42, 117, 36]'}
{'accuracy': '0/1 : 0.0%', 'predicted': '[7, 19, 14, 5, 12, 9, 8, 13, 18, 15, 1, 6, 11, 8, 16, 17, 4, 10]', 'actual': '[37, 133, 80, 142, 100, 20, 125, 28, 147, 94, 82, 17, 101, 43, 92, 42, 117, 36]'}




#################### Running 




{'accuracy': '0/2 : 0.0%', 'predicted': '[5, 10, 13, 7, 16, 3, 12, 9, 5, 15, 8, 14, 4, 9, 11, 6, 2, 1]', 'actual': '[42, 109, 42, 42, 105, 145, 34, 110, 42, 53, 136, 111, 125, 142, 32, 10, 42, 95]'}
{'accuracy': '0/2 : 0.0%', 'predicted': '[5, 10, 13, 7, 16, 3, 12, 9, 5, 15, 8, 14, 4, 9, 11, 6, 2, 1]', 'actual': '[42, 109, 42, 42, 105, 145, 34, 110, 42, 53, 136, 111, 125, 142, 32, 10, 42, 95]'}


...



#################### Running 




{'accuracy': '0/49 : 0.0%', 'predicted': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 8, 15, 16, 17, 18, 19, 20, 6]', 'actual': '[42, 145, 21, 81, 17, 58, 141, 123, 34, 85, 112, 94, 126, 123, 28, 54, 102, 35, 32, 11, 58]'}
{'accuracy': '0/49 : 0.0%', 'predicted': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 8, 15, 16, 17, 18, 19, 20, 6]', 'actual': '[42, 145, 21, 81, 17, 58, 141, 123, 34, 85, 112, 94, 126, 123, 28, 54, 102, 35, 32, 11, 58]'}




#################### Running 




{'accuracy': '0/50 : 0.0%', 'predicted': '[38, 72, 15, 57, 84, 33, 29, 41, 95, 67]', 'actual': '[126, 0, 27, 124, 42, 144, 14, 76, 42, 85]'}
{'accuracy': '0/50 : 0.0%', 'predicted': '[38, 72, 15, 57, 84, 33, 29, 41, 95, 67]', 'actual': '[126, 0, 27, 124, 42, 144, 14, 76, 42, 85]'}
Time taken for evaluation: 161.3048689365387 sec
Time taken for evaluation: 161.3048689365387 sec


Final Accuracy: 0.0
